최근 1년간 발표된 '이미지','음성','강화학습','텍스트' 논문들에서 가장 많이 나온 단어를 찾고, 각각 '영상','음성','강화학습','텍스트'와 유사도가 높은 단어를 추출합니다


data수집:
DBpia에서 '이미지','음성','강화학습','텍스트' 를 검색해서 나온 논문의 초록을 크롤링합니당!


In [1]:
from bs4 import BeautifulSoup as bs
import selenium.webdriver as webdriver
from urllib.request import Request, urlopen
from time import sleep
import pandas as pd

### DBpia 로그인

In [32]:
my_url = "https://www.dbpia.co.kr/member/b2bLogin"
browser = webdriver.Chrome("./chromedriver.exe")
browser.get(my_url)

In [33]:
browser.find_element_by_xpath('//*[@id="dev_search"]').send_keys('광운대학교')
sleep(2)
browser.find_element_by_xpath('//*[@id="dev_searchBtn"]/span').click()
sleep(2)
browser.find_element_by_xpath('//*[@id="ICST00000449"]').click()

In [34]:
browser.find_element_by_name('b2bAcc').send_keys('2016707072')#아이디 입력
browser.find_element_by_name('userPass').send_keys('tlsalswjd9!')#비밀번호 입력
browser.find_element_by_xpath('//*[@id="dev_loginBtn"]').click()#인증버튼 클릭
browser.find_element_by_xpath('/html/body/div[6]/div[3]/div/button').click()#인증완료 버튼 클릭


### 분류 별 최근1년 논문 초록 크롤링

In [35]:
keys = ["영상", "음성", "강화학습", "텍스트"]
texts = []
for i, key in enumerate(keys):
    browser.find_element_by_xpath('//*[@id="header"]/div[4]/div[2]/h1/a').click()
    browser.find_element_by_xpath('//*[@id="keyword"]').send_keys(key) 
    browser.find_element_by_xpath('//*[@id="header"]/div[4]/div[2]/div[1]/div[1]/a').click()
    sleep(2)
    #"영상", "음성", "강화학습", "텍스트" 차례로 입력 후 검색
    
    
    #공학 카테고리 클릭(영상,음성은 첫번째 카테고리, 강화학습은 두번째, 텍스트는 세번째 카테고리)
    if (i == 0) or (i == 1) :
        browser.find_element_by_xpath('//*[@id="dev_category"]/li[1]/span/label').click()
    elif i == 2 :
        browser.find_element_by_xpath('//*[@id="dev_category"]/li[2]/span/label').click()
    else :
        browser.find_element_by_xpath('//*[@id="dev_category"]/li[3]/span/label').click()
        
    sleep(1)
    
    
    browser.find_element_by_xpath('//*[@id="last_year_1"]').click()#최근 1년 논문만
        
    
    for n in range(2):
        browser.find_element_by_xpath('//*[@id="contents"]/div[2]/div[3]/div[3]/div[3]/div/a/span').click()
        sleep(2)
    
    # 더 많은 논문을 불러오기 위해 더보기 2번 클릭
    body_source = bs(browser.page_source, "html.parser")
    url_table = body_source.find_all('div', {'class': 'titWrap'})
    #print(url_table)

    url_contents = []
    for url in url_table:
        url_source = 'https://www.dbpia.co.kr'+url.find('a', target = '_blank').get("href")
        url_contents.append(url_source)
    url_contents = url_contents[:-2]#뒤에 두개 이상한 path 버림!
    
    #print(key,len(url_contents))

    text_key = []
    for j, url in enumerate(url_contents):
        browser.get(url)
        browser.implicitly_wait(20)
        body_source = bs(browser.page_source, "html.parser")
        
        #texts[i].append(text)
        #text = body_source.find('p', class_="article").text
        try :
            text = body_source.find('p', class_="article").text
            
            #print(text)
            if '다' in text: #한글초록만 보겠습니당~
                text_key.append(text)
        except :
            pass
    print(key, len(text_key))
    texts.append(text_key)

영상 28
음성 21
강화학습 13
텍스트 16


##### '영상'으로 검색해서 크롤링한 초록 28개 (영어, None 제외)

In [40]:
len(texts[0])

28

In [36]:
texts[0] # 영상으로 검색한 논문의 초록 모음

['최근 UAV (Unmanned Aerial Vehicle)를 이용하여 고해상도 영상을 편리하게 취득할 수 있게 되면서 저비용으로 소규모 지역의 관측 및 공간정보 제작이 가능하게 되었다. 특히, 농업환경 모니터링을 위하여 작물생산 지역의 피복지도 생성에 대한 연구가 활발히 진행되고 있으며, 랜덤 포레스트와 SVM (Support Vector Machine) 및 CNN (Convolutional Neural Network) 을 적용하여 분류 성능을 비교한 결과 영상분류에서 딥러닝 적용에 대하여 활용도가 높은 것으로 나타났다. 특히, 위성영상을 이용한 피복분류는 위성영상 데이터 셋과 선행 파라메터를 사용하여 피복분류의 정확도와 시간에 대한 장점을 가지고 있다. 하지만, 무인항공기 영상은 위성영상과 공간해상도와 같은 특성이 달라 이를 적용하기에는 어려움이 있다. 이러한 문제점을 해결하기 위하여 위성영상 데이터 셋이 아닌 UAV를 이용한 데이터 셋과 국내의 소규모 복합 피복이 존재하는 농경지 분석에 활용이 가능한 딥러닝 알고리즘 적용 연구를 수행하였다. 본 연구에서는 최신 딥러닝의 의미론적 영상분류인 DeepLab V3+, FC-DenseNet (Fully Convolutional DenseNets) , FRRN-B (Full-Resolution Residual Networks) 를 UAV 데이터 셋에 적용하여 영상분류를 수행하였다. 분류 결과 DeepLab V3+와 FC-DenseNet의 적용 결과가 기존 감독분류보다 높은 전체 정확도 97%, Kappa 계수 0.92로 소규모 지역의 UAV 영상을 활용한 피복분류의 적용가능성을 보여주었다.',
 '흉부 영상을 통해 진단 가능한 병변은 무기폐, 심비대, 덩어리, 기흉, 삼출 등 그 종류가 수십 가지에 이른다. 흉부병변의 정확한 진단과 위치 및 크기를 판단하기 위해 일반적으로 전산화단층촬영(CT) 검사가 필요하지만, 전산화단층촬영은 검사 비용과 방사선 피폭 등의 단점이 있다. 따라서 본 논문에서는 흉부 병변 진단의 

In [44]:
text_img = " ".join(texts[0]) # 초록을 하나의 문자열로 붙이기
text_img

'최근 UAV (Unmanned Aerial Vehicle)를 이용하여 고해상도 영상을 편리하게 취득할 수 있게 되면서 저비용으로 소규모 지역의 관측 및 공간정보 제작이 가능하게 되었다. 특히, 농업환경 모니터링을 위하여 작물생산 지역의 피복지도 생성에 대한 연구가 활발히 진행되고 있으며, 랜덤 포레스트와 SVM (Support Vector Machine) 및 CNN (Convolutional Neural Network) 을 적용하여 분류 성능을 비교한 결과 영상분류에서 딥러닝 적용에 대하여 활용도가 높은 것으로 나타났다. 특히, 위성영상을 이용한 피복분류는 위성영상 데이터 셋과 선행 파라메터를 사용하여 피복분류의 정확도와 시간에 대한 장점을 가지고 있다. 하지만, 무인항공기 영상은 위성영상과 공간해상도와 같은 특성이 달라 이를 적용하기에는 어려움이 있다. 이러한 문제점을 해결하기 위하여 위성영상 데이터 셋이 아닌 UAV를 이용한 데이터 셋과 국내의 소규모 복합 피복이 존재하는 농경지 분석에 활용이 가능한 딥러닝 알고리즘 적용 연구를 수행하였다. 본 연구에서는 최신 딥러닝의 의미론적 영상분류인 DeepLab V3+, FC-DenseNet (Fully Convolutional DenseNets) , FRRN-B (Full-Resolution Residual Networks) 를 UAV 데이터 셋에 적용하여 영상분류를 수행하였다. 분류 결과 DeepLab V3+와 FC-DenseNet의 적용 결과가 기존 감독분류보다 높은 전체 정확도 97%, Kappa 계수 0.92로 소규모 지역의 UAV 영상을 활용한 피복분류의 적용가능성을 보여주었다. 흉부 영상을 통해 진단 가능한 병변은 무기폐, 심비대, 덩어리, 기흉, 삼출 등 그 종류가 수십 가지에 이른다. 흉부병변의 정확한 진단과 위치 및 크기를 판단하기 위해 일반적으로 전산화단층촬영(CT) 검사가 필요하지만, 전산화단층촬영은 검사 비용과 방사선 피폭 등의 단점이 있다. 따라서 본 논문에서는 흉부 병변 진단의 일차적 선

In [48]:
type(text_img)

str

In [67]:
img = text_img.split('.')
#문장단위로 나누기
print(len(img))
img

171


['최근 UAV (Unmanned Aerial Vehicle)를 이용하여 고해상도 영상을 편리하게 취득할 수 있게 되면서 저비용으로 소규모 지역의 관측 및 공간정보 제작이 가능하게 되었다',
 ' 특히, 농업환경 모니터링을 위하여 작물생산 지역의 피복지도 생성에 대한 연구가 활발히 진행되고 있으며, 랜덤 포레스트와 SVM (Support Vector Machine) 및 CNN (Convolutional Neural Network) 을 적용하여 분류 성능을 비교한 결과 영상분류에서 딥러닝 적용에 대하여 활용도가 높은 것으로 나타났다',
 ' 특히, 위성영상을 이용한 피복분류는 위성영상 데이터 셋과 선행 파라메터를 사용하여 피복분류의 정확도와 시간에 대한 장점을 가지고 있다',
 ' 하지만, 무인항공기 영상은 위성영상과 공간해상도와 같은 특성이 달라 이를 적용하기에는 어려움이 있다',
 ' 이러한 문제점을 해결하기 위하여 위성영상 데이터 셋이 아닌 UAV를 이용한 데이터 셋과 국내의 소규모 복합 피복이 존재하는 농경지 분석에 활용이 가능한 딥러닝 알고리즘 적용 연구를 수행하였다',
 ' 본 연구에서는 최신 딥러닝의 의미론적 영상분류인 DeepLab V3+, FC-DenseNet (Fully Convolutional DenseNets) , FRRN-B (Full-Resolution Residual Networks) 를 UAV 데이터 셋에 적용하여 영상분류를 수행하였다',
 ' 분류 결과 DeepLab V3+와 FC-DenseNet의 적용 결과가 기존 감독분류보다 높은 전체 정확도 97%, Kappa 계수 0',
 '92로 소규모 지역의 UAV 영상을 활용한 피복분류의 적용가능성을 보여주었다',
 ' 흉부 영상을 통해 진단 가능한 병변은 무기폐, 심비대, 덩어리, 기흉, 삼출 등 그 종류가 수십 가지에 이른다',
 ' 흉부병변의 정확한 진단과 위치 및 크기를 판단하기 위해 일반적으로 전산화단층촬영(CT) 검사가 필요하지만, 전산화단층촬영은 검사 비용과 방사선 피폭 등

##### '음성'으로 검색해서 크롤링한 초록 21개 (영어, None 제외)

In [37]:
texts[1] # 음성으로 검색한 논문의 초록 모음

['스마트폰은 시각장애인들에게 단순한 통신수단으로서의 의미를 넘어 신체적 제약을 극복할 수 있는 유용한 도구로 인식되고 있다. 터치스크린 기반의 스마트폰이 시각장애인에게 이와 같은 의미가 된 것은 최근 급격히 발전한 인공지능 기술 덕분이라고 할 수 있다. 본 연구에서는 시각장애인, 특히 저시력인에게 온라인 및 오프라인 정보 접근성을 제공하기 위한 앱 솔루션을 제안한다. 모든 서비스의 구동과 결과 전달 인터페이스는 음성기반이며 사전 및 지식 검색이 가능하고 주변 물체를 파악하여 알려주는 이미지 캡셔닝 기능과 작은 글씨를 읽어 주는 OCR 기능 등을 탑재한다. 실시간 검색 순위 정보를 전할 때, 그에 대한 네티즌의 반응도 함께 전달한다. 이 앱은 저시력인 뿐 아니라 시력이 약한 노인분들과 안과 질환에서 회복 중인 환자 등에게도 활용될 수 있다.',
 '청각장애인은 말소리에서 느껴지는 비언어적 정보를 인식하기 어려우며, 이러한 말소리와 글자 간의 간극으로 인해 글자에 대한 흥미를 갖기 어렵다. 이와 같은 어려움을 인식하고 본 연구에서는 시각적 자극에 대한 말소리의 정보 인식을 알아보고자 하였다. 말소리의 특징을 시각적으로 묘사한 여러 글자의 움직임, 즉 키네틱 타이포그래피를 보여주고 이에 대한 청각장애인의 반응을 심층 조사하였으며 말소리의 음성과 감정 정보를 어떻게 인식하는지 알아보았다. 조사 결과로 청각장애인은 각기 다른 글자 움직임을 통해 목소리 간의 음성과 감정 차이를 분명히 인식할 수 있으며 여러 키네틱 타이포그래피 표현 중 목소리의 높고 낮음에 대한 글자의 수직적 위치 변화에서 음성 정보와 감정 정보를 가장 정확히 인식한다고 나타났다.',
 '현재 우리나라에서 난청 환자에게 디지털 보청기의 삽입 이득을 피팅할 때 사용하고 있는 알고리즘은 거의 외국에서 개발된 것을 그대로 사용하고 있다. 기존의 피팅 알고리즘 중, 음성의 명료도에 중점을 두어 개발된 것이 NAL-NL2 알고리즘이다. 이 알고리즘은 영어의 어음 명료도를 기준으로 개발된 것이기 때문에 우리나라 

In [49]:
text_voice = " ".join(texts[1]) # 초록을 하나의 문자열로 붙이기
text_voice

'스마트폰은 시각장애인들에게 단순한 통신수단으로서의 의미를 넘어 신체적 제약을 극복할 수 있는 유용한 도구로 인식되고 있다. 터치스크린 기반의 스마트폰이 시각장애인에게 이와 같은 의미가 된 것은 최근 급격히 발전한 인공지능 기술 덕분이라고 할 수 있다. 본 연구에서는 시각장애인, 특히 저시력인에게 온라인 및 오프라인 정보 접근성을 제공하기 위한 앱 솔루션을 제안한다. 모든 서비스의 구동과 결과 전달 인터페이스는 음성기반이며 사전 및 지식 검색이 가능하고 주변 물체를 파악하여 알려주는 이미지 캡셔닝 기능과 작은 글씨를 읽어 주는 OCR 기능 등을 탑재한다. 실시간 검색 순위 정보를 전할 때, 그에 대한 네티즌의 반응도 함께 전달한다. 이 앱은 저시력인 뿐 아니라 시력이 약한 노인분들과 안과 질환에서 회복 중인 환자 등에게도 활용될 수 있다. 청각장애인은 말소리에서 느껴지는 비언어적 정보를 인식하기 어려우며, 이러한 말소리와 글자 간의 간극으로 인해 글자에 대한 흥미를 갖기 어렵다. 이와 같은 어려움을 인식하고 본 연구에서는 시각적 자극에 대한 말소리의 정보 인식을 알아보고자 하였다. 말소리의 특징을 시각적으로 묘사한 여러 글자의 움직임, 즉 키네틱 타이포그래피를 보여주고 이에 대한 청각장애인의 반응을 심층 조사하였으며 말소리의 음성과 감정 정보를 어떻게 인식하는지 알아보았다. 조사 결과로 청각장애인은 각기 다른 글자 움직임을 통해 목소리 간의 음성과 감정 차이를 분명히 인식할 수 있으며 여러 키네틱 타이포그래피 표현 중 목소리의 높고 낮음에 대한 글자의 수직적 위치 변화에서 음성 정보와 감정 정보를 가장 정확히 인식한다고 나타났다. 현재 우리나라에서 난청 환자에게 디지털 보청기의 삽입 이득을 피팅할 때 사용하고 있는 알고리즘은 거의 외국에서 개발된 것을 그대로 사용하고 있다. 기존의 피팅 알고리즘 중, 음성의 명료도에 중점을 두어 개발된 것이 NAL-NL2 알고리즘이다. 이 알고리즘은 영어의 어음 명료도를 기준으로 개발된 것이기 때문에 우리나라 난청 환자들에게 

In [88]:
voice = text_voice.split('.')
#문장단위로 나누기
print(len(voice))
voice

124


['스마트폰은 시각장애인들에게 단순한 통신수단으로서의 의미를 넘어 신체적 제약을 극복할 수 있는 유용한 도구로 인식되고 있다',
 ' 터치스크린 기반의 스마트폰이 시각장애인에게 이와 같은 의미가 된 것은 최근 급격히 발전한 인공지능 기술 덕분이라고 할 수 있다',
 ' 본 연구에서는 시각장애인, 특히 저시력인에게 온라인 및 오프라인 정보 접근성을 제공하기 위한 앱 솔루션을 제안한다',
 ' 모든 서비스의 구동과 결과 전달 인터페이스는 음성기반이며 사전 및 지식 검색이 가능하고 주변 물체를 파악하여 알려주는 이미지 캡셔닝 기능과 작은 글씨를 읽어 주는 OCR 기능 등을 탑재한다',
 ' 실시간 검색 순위 정보를 전할 때, 그에 대한 네티즌의 반응도 함께 전달한다',
 ' 이 앱은 저시력인 뿐 아니라 시력이 약한 노인분들과 안과 질환에서 회복 중인 환자 등에게도 활용될 수 있다',
 ' 청각장애인은 말소리에서 느껴지는 비언어적 정보를 인식하기 어려우며, 이러한 말소리와 글자 간의 간극으로 인해 글자에 대한 흥미를 갖기 어렵다',
 ' 이와 같은 어려움을 인식하고 본 연구에서는 시각적 자극에 대한 말소리의 정보 인식을 알아보고자 하였다',
 ' 말소리의 특징을 시각적으로 묘사한 여러 글자의 움직임, 즉 키네틱 타이포그래피를 보여주고 이에 대한 청각장애인의 반응을 심층 조사하였으며 말소리의 음성과 감정 정보를 어떻게 인식하는지 알아보았다',
 ' 조사 결과로 청각장애인은 각기 다른 글자 움직임을 통해 목소리 간의 음성과 감정 차이를 분명히 인식할 수 있으며 여러 키네틱 타이포그래피 표현 중 목소리의 높고 낮음에 대한 글자의 수직적 위치 변화에서 음성 정보와 감정 정보를 가장 정확히 인식한다고 나타났다',
 ' 현재 우리나라에서 난청 환자에게 디지털 보청기의 삽입 이득을 피팅할 때 사용하고 있는 알고리즘은 거의 외국에서 개발된 것을 그대로 사용하고 있다',
 ' 기존의 피팅 알고리즘 중, 음성의 명료도에 중점을 두어 개발된 것이 NAL-NL2 알고리즘이다',
 ' 이 

##### '강화학습'으로 검색해서 크롤링한 초록 13개 (영어, None 제외)

In [38]:
texts[2] # 강화학습으로 검색한 논문의 초록 모음

['딥러닝 알고리즘의 발전에 따라 강화학습은 게임이나 물리 기반 모델과 같이 연속적인 동작을 필요로 하는 많은 분야에서 큰 활약을 하고 있다. 이에 따라 OpenAI의 Gym모듈과 같이 여러 강화학습 알고리즘을 실험하고 비교하기 위한 여러가지 방법들이 고안 되었고 강화학습을 시각화하고 분석하는데 많은 기여를 하였다. 현재 센서에 기반한 규칙 기반 시스템을 따르고 있는 많은 자율주차 알고리즘과 차량의 자율성에 대한 수요가 증가하는 것을 고려했을 때, 자율주차에 강화학습을 적용하는 연구의 필요성이 대두되고 있고, 이를 위한 시뮬레이터가 필요하다. 본 논문은 강화학습 연구가 가능한 자율주차 시뮬레이터를 개발한다. 실제 차량의 동작 방식을 모델링하기 위하여 애커만 조향 기하학 모델 수식에 기반한 자율주차 시뮬레이터를 개발한 뒤, Deep Deterministic Policy Gradient (DDPG) 강화학습 알고리즘을 적용하여 개발한 자율주차 시뮬레이터가 성공적으로 학습하는 모습을 보여주었다.',
 '연구는 MOBA 게임에서 선호도가 낮은 Supporter를 대체하는 인공지능을 강화학습을 이용한 구현을 목표하였다. ML_Agent를 이용해 게임의 규칙, 환경, 관측 정보, 보상 처벌을 구성하였다. DPS 에이전트로 구성된 그룹과, Support 에이전트가 있는 그룹으로 나누어 강화학습을 진행하였다. 결과 데이터인 누적 보상 값, 사망 횟수 바탕으로 결론을 도출하였다. 협력 플레이 그룹이 비교 그룹보다 평균 누적 보상 값이 3.3 더 높게 측정되었으며 사망 횟수 총합 평균은 3.15 낮게 되었다. 이를 바탕으로 죽음을 최소화하고 보상을 최대화하는 협력 플레이를 수행하는 강화학습을 확인할 수 있었다.',
 '스마트 팩토리에서 사용되는 로봇들은 대부분 사용자가 원격 조종하거나, 라인 트레이서 기법으로 대부분 출시되어 상용화 되었다. 이 로봇들은 지정된 공정 위치 또는 알람 경보가 작동된 위치로 이동하여 부속품을 전달하거나 수리를 하는 데 사용된다. 하지만 이러한 방법들

In [50]:
text_RL = " ".join(texts[2]) # 초록을 하나의 문자열로 붙이기
text_RL

'딥러닝 알고리즘의 발전에 따라 강화학습은 게임이나 물리 기반 모델과 같이 연속적인 동작을 필요로 하는 많은 분야에서 큰 활약을 하고 있다. 이에 따라 OpenAI의 Gym모듈과 같이 여러 강화학습 알고리즘을 실험하고 비교하기 위한 여러가지 방법들이 고안 되었고 강화학습을 시각화하고 분석하는데 많은 기여를 하였다. 현재 센서에 기반한 규칙 기반 시스템을 따르고 있는 많은 자율주차 알고리즘과 차량의 자율성에 대한 수요가 증가하는 것을 고려했을 때, 자율주차에 강화학습을 적용하는 연구의 필요성이 대두되고 있고, 이를 위한 시뮬레이터가 필요하다. 본 논문은 강화학습 연구가 가능한 자율주차 시뮬레이터를 개발한다. 실제 차량의 동작 방식을 모델링하기 위하여 애커만 조향 기하학 모델 수식에 기반한 자율주차 시뮬레이터를 개발한 뒤, Deep Deterministic Policy Gradient (DDPG) 강화학습 알고리즘을 적용하여 개발한 자율주차 시뮬레이터가 성공적으로 학습하는 모습을 보여주었다. 연구는 MOBA 게임에서 선호도가 낮은 Supporter를 대체하는 인공지능을 강화학습을 이용한 구현을 목표하였다. ML_Agent를 이용해 게임의 규칙, 환경, 관측 정보, 보상 처벌을 구성하였다. DPS 에이전트로 구성된 그룹과, Support 에이전트가 있는 그룹으로 나누어 강화학습을 진행하였다. 결과 데이터인 누적 보상 값, 사망 횟수 바탕으로 결론을 도출하였다. 협력 플레이 그룹이 비교 그룹보다 평균 누적 보상 값이 3.3 더 높게 측정되었으며 사망 횟수 총합 평균은 3.15 낮게 되었다. 이를 바탕으로 죽음을 최소화하고 보상을 최대화하는 협력 플레이를 수행하는 강화학습을 확인할 수 있었다. 스마트 팩토리에서 사용되는 로봇들은 대부분 사용자가 원격 조종하거나, 라인 트레이서 기법으로 대부분 출시되어 상용화 되었다. 이 로봇들은 지정된 공정 위치 또는 알람 경보가 작동된 위치로 이동하여 부속품을 전달하거나 수리를 하는 데 사용된다. 하지만 이러한 방법들은 수시로 변하는

In [89]:
RL = text_RL.split('.')
#문장단위로 나누기
print(len(RL))
RL

83


['딥러닝 알고리즘의 발전에 따라 강화학습은 게임이나 물리 기반 모델과 같이 연속적인 동작을 필요로 하는 많은 분야에서 큰 활약을 하고 있다',
 ' 이에 따라 OpenAI의 Gym모듈과 같이 여러 강화학습 알고리즘을 실험하고 비교하기 위한 여러가지 방법들이 고안 되었고 강화학습을 시각화하고 분석하는데 많은 기여를 하였다',
 ' 현재 센서에 기반한 규칙 기반 시스템을 따르고 있는 많은 자율주차 알고리즘과 차량의 자율성에 대한 수요가 증가하는 것을 고려했을 때, 자율주차에 강화학습을 적용하는 연구의 필요성이 대두되고 있고, 이를 위한 시뮬레이터가 필요하다',
 ' 본 논문은 강화학습 연구가 가능한 자율주차 시뮬레이터를 개발한다',
 ' 실제 차량의 동작 방식을 모델링하기 위하여 애커만 조향 기하학 모델 수식에 기반한 자율주차 시뮬레이터를 개발한 뒤, Deep Deterministic Policy Gradient (DDPG) 강화학습 알고리즘을 적용하여 개발한 자율주차 시뮬레이터가 성공적으로 학습하는 모습을 보여주었다',
 ' 연구는 MOBA 게임에서 선호도가 낮은 Supporter를 대체하는 인공지능을 강화학습을 이용한 구현을 목표하였다',
 ' ML_Agent를 이용해 게임의 규칙, 환경, 관측 정보, 보상 처벌을 구성하였다',
 ' DPS 에이전트로 구성된 그룹과, Support 에이전트가 있는 그룹으로 나누어 강화학습을 진행하였다',
 ' 결과 데이터인 누적 보상 값, 사망 횟수 바탕으로 결론을 도출하였다',
 ' 협력 플레이 그룹이 비교 그룹보다 평균 누적 보상 값이 3',
 '3 더 높게 측정되었으며 사망 횟수 총합 평균은 3',
 '15 낮게 되었다',
 ' 이를 바탕으로 죽음을 최소화하고 보상을 최대화하는 협력 플레이를 수행하는 강화학습을 확인할 수 있었다',
 ' 스마트 팩토리에서 사용되는 로봇들은 대부분 사용자가 원격 조종하거나, 라인 트레이서 기법으로 대부분 출시되어 상용화 되었다',
 ' 이 로봇들은 지정된 공정 위치 또는 알람 경보가 작동

##### '텍스트'으로 검색해서 크롤링한 초록 16개 (영어, None 제외)

In [39]:
texts[3] # 텍스트로 검색한 논문의 초록 모음

['본 연구에서는 \n 글파일의 보다 효율적인 활용성을 위해 E-mail 에 첨부된 표와 텍스트 기반의 \n 글 파일을 웹브라우저에서 바로 확인할 수 있도록 \n 글 파일을 HTML파일로 변환하는 소프트웨어를 구현하였다. 관공서나 공공기관에서는 내부 결재 및 공문을 \n 글로 작성하고, 작성된 문서를 게시판이나 E-mail을 통해 공유하고 있다. 이런 문서는 대부분 표와 텍스트로 구성되어 있고 대부분 편집이 아닌 열람을 목적으로 웹에서 공유되고 있다. 이러한 사용목적에 효율적인 변환 소프트웨어를 구현하기 위해 기존에 존재하는 변환 소프트웨어의 실행적 번거로움과 변환 오버헤드를 줄이는 방법을 제시한다. 표와 텍스트의 특성을 이용하여 서식파일의 패턴을 사용하고, 변환 후 구조를 단순화 하도록 구현하였다. 본 논문에서 제시한 방법으로 변환된 문서와 기존의 변환기에서 생성된 HTML 문서와 비교했을 때, 태그 속성의 사용이 대폭 감소되었고, 변화 후 2개의 파일만 생성되어 변환 결과가 단순화되었음을 보여주었다.',
 '본 연구에서는 소셜 네트워크 서비스(Social Network Service, SNS)인 플리커(Flickr)에 게시된 지오태깅된 사진 데이터와 텍스트 데이터를 활용하여 우리나라 방문객이 갖는 지역별 이미지를 비교하였다. 플리커 데이터 수집은 2013년부터 2018년까지 약 6년간 데이터를 수집하였으며, 6년간 수집된 데이터 29만여건 가운데 관광객이 업로드한 것으로 추정되는 약 17만장의 사진을 분석대상으로 하였다. 플리커의 태그와 텍스트에 대한 분석은 텍스트마이닝 기법 중 단어간 빈도분석 및 연관관계분석을 적용하였으며, 사진에 대한 분석은 합성곱 신경망(Convolutional Neural Network, CNN) 중 하나인 Inception V3 모델을 활용하였다. 지역에 대한 이미지 분석은 서울, 부산, 제주 지역을 대상으로 하였으며, 텍스트 분석 결과와 사진 이미지 분석결과를 비교하여 지역 이미지의 차이를 분석하였다.',
 '지속적으로 새롭게 

In [51]:
text_txt = " ".join(texts[3]) # 초록을 하나의 문자열로 붙이기
text_txt

'본 연구에서는 \n 글파일의 보다 효율적인 활용성을 위해 E-mail 에 첨부된 표와 텍스트 기반의 \n 글 파일을 웹브라우저에서 바로 확인할 수 있도록 \n 글 파일을 HTML파일로 변환하는 소프트웨어를 구현하였다. 관공서나 공공기관에서는 내부 결재 및 공문을 \n 글로 작성하고, 작성된 문서를 게시판이나 E-mail을 통해 공유하고 있다. 이런 문서는 대부분 표와 텍스트로 구성되어 있고 대부분 편집이 아닌 열람을 목적으로 웹에서 공유되고 있다. 이러한 사용목적에 효율적인 변환 소프트웨어를 구현하기 위해 기존에 존재하는 변환 소프트웨어의 실행적 번거로움과 변환 오버헤드를 줄이는 방법을 제시한다. 표와 텍스트의 특성을 이용하여 서식파일의 패턴을 사용하고, 변환 후 구조를 단순화 하도록 구현하였다. 본 논문에서 제시한 방법으로 변환된 문서와 기존의 변환기에서 생성된 HTML 문서와 비교했을 때, 태그 속성의 사용이 대폭 감소되었고, 변화 후 2개의 파일만 생성되어 변환 결과가 단순화되었음을 보여주었다. 본 연구에서는 소셜 네트워크 서비스(Social Network Service, SNS)인 플리커(Flickr)에 게시된 지오태깅된 사진 데이터와 텍스트 데이터를 활용하여 우리나라 방문객이 갖는 지역별 이미지를 비교하였다. 플리커 데이터 수집은 2013년부터 2018년까지 약 6년간 데이터를 수집하였으며, 6년간 수집된 데이터 29만여건 가운데 관광객이 업로드한 것으로 추정되는 약 17만장의 사진을 분석대상으로 하였다. 플리커의 태그와 텍스트에 대한 분석은 텍스트마이닝 기법 중 단어간 빈도분석 및 연관관계분석을 적용하였으며, 사진에 대한 분석은 합성곱 신경망(Convolutional Neural Network, CNN) 중 하나인 Inception V3 모델을 활용하였다. 지역에 대한 이미지 분석은 서울, 부산, 제주 지역을 대상으로 하였으며, 텍스트 분석 결과와 사진 이미지 분석결과를 비교하여 지역 이미지의 차이를 분석하였다. 지속적으로 새롭게 등장하는 악성 파

In [90]:
txt = text_txt.split('.')
#문장단위로 나누기
print(len(txt))
txt

100


['본 연구에서는 \n 글파일의 보다 효율적인 활용성을 위해 E-mail 에 첨부된 표와 텍스트 기반의 \n 글 파일을 웹브라우저에서 바로 확인할 수 있도록 \n 글 파일을 HTML파일로 변환하는 소프트웨어를 구현하였다',
 ' 관공서나 공공기관에서는 내부 결재 및 공문을 \n 글로 작성하고, 작성된 문서를 게시판이나 E-mail을 통해 공유하고 있다',
 ' 이런 문서는 대부분 표와 텍스트로 구성되어 있고 대부분 편집이 아닌 열람을 목적으로 웹에서 공유되고 있다',
 ' 이러한 사용목적에 효율적인 변환 소프트웨어를 구현하기 위해 기존에 존재하는 변환 소프트웨어의 실행적 번거로움과 변환 오버헤드를 줄이는 방법을 제시한다',
 ' 표와 텍스트의 특성을 이용하여 서식파일의 패턴을 사용하고, 변환 후 구조를 단순화 하도록 구현하였다',
 ' 본 논문에서 제시한 방법으로 변환된 문서와 기존의 변환기에서 생성된 HTML 문서와 비교했을 때, 태그 속성의 사용이 대폭 감소되었고, 변화 후 2개의 파일만 생성되어 변환 결과가 단순화되었음을 보여주었다',
 ' 본 연구에서는 소셜 네트워크 서비스(Social Network Service, SNS)인 플리커(Flickr)에 게시된 지오태깅된 사진 데이터와 텍스트 데이터를 활용하여 우리나라 방문객이 갖는 지역별 이미지를 비교하였다',
 ' 플리커 데이터 수집은 2013년부터 2018년까지 약 6년간 데이터를 수집하였으며, 6년간 수집된 데이터 29만여건 가운데 관광객이 업로드한 것으로 추정되는 약 17만장의 사진을 분석대상으로 하였다',
 ' 플리커의 태그와 텍스트에 대한 분석은 텍스트마이닝 기법 중 단어간 빈도분석 및 연관관계분석을 적용하였으며, 사진에 대한 분석은 합성곱 신경망(Convolutional Neural Network, CNN) 중 하나인 Inception V3 모델을 활용하였다',
 ' 지역에 대한 이미지 분석은 서울, 부산, 제주 지역을 대상으로 하였으며, 텍스트 분석 결과와 사진 이미지 분석결과를 비교하여 지

## Tokenizing

In [10]:
import numpy as np
import pandas as pd
!pip install KoNLPy
from konlpy.tag import *
from gensim.models import Word2Vec, fasttext


In [11]:
twitter = Okt()  #twitter class 객체당!
komoran = Komoran()
kkma = Kkma()
# mecab = Mecab()
# hannanum = Hannanum()

'영상'으로 검색한 논문의 초록으로 Tokenizing을 해보자

In [55]:
#twitter
print("twitter", len(twitter.pos(text_img)))
print(twitter.pos(text_img))

twitter 5488
[('최근', 'Noun'), ('UAV', 'Alpha'), ('(', 'Punctuation'), ('Unmanned', 'Alpha'), ('Aerial', 'Alpha'), ('Vehicle', 'Alpha'), (')', 'Punctuation'), ('를', 'Noun'), ('이용', 'Noun'), ('하여', 'Verb'), ('고', 'Modifier'), ('해상도', 'Noun'), ('영상', 'Noun'), ('을', 'Josa'), ('편리하게', 'Adjective'), ('취득', 'Noun'), ('할', 'Verb'), ('수', 'Noun'), ('있게', 'Adjective'), ('되면서', 'Verb'), ('저', 'Determiner'), ('비용', 'Noun'), ('으로', 'Josa'), ('소규모', 'Noun'), ('지역', 'Noun'), ('의', 'Josa'), ('관측', 'Noun'), ('및', 'Noun'), ('공간', 'Noun'), ('정보', 'Noun'), ('제작', 'Noun'), ('이', 'Josa'), ('가능하게', 'Adjective'), ('되었다', 'Verb'), ('.', 'Punctuation'), ('특히', 'Adverb'), (',', 'Punctuation'), ('농업', 'Noun'), ('환경', 'Noun'), ('모니터링', 'Noun'), ('을', 'Josa'), ('위', 'Noun'), ('하여', 'Verb'), ('작물', 'Noun'), ('생산', 'Noun'), ('지역', 'Noun'), ('의', 'Josa'), ('피복', 'Noun'), ('지도', 'Noun'), ('생', 'Noun'), ('성', 'Suffix'), ('에', 'Josa'), ('대한', 'Noun'), ('연구', 'Noun'), ('가', 'Josa'), ('활발히', 'Adjective'), ('진행', 'Noun'), (

In [56]:
#komoran
print("komoran", len(komoran.pos(text_img)))
print(komoran.pos(text_img))

komoran 6619
[('최근', 'NNG'), ('UAV', 'SL'), ('(', 'SS'), ('Unmanned', 'SL'), ('Aerial', 'SL'), ('Vehicle', 'SL'), (')', 'SS'), ('를', 'JKO'), ('이용', 'NNG'), ('하', 'XSV'), ('아', 'EC'), ('고', 'XPN'), ('해상도', 'NNP'), ('영상', 'NNG'), ('을', 'JKO'), ('편리', 'NNG'), ('하', 'XSV'), ('게', 'EC'), ('취득', 'NNG'), ('하', 'XSV'), ('ㄹ', 'ETM'), ('수', 'NNB'), ('있', 'VV'), ('게', 'EC'), ('되', 'VV'), ('면서', 'EC'), ('저', 'MM'), ('비용', 'NNP'), ('으로', 'JKB'), ('소', 'XPN'), ('규모', 'NNG'), ('지역', 'NNG'), ('의', 'JKG'), ('관측', 'NNP'), ('및', 'MAJ'), ('공간', 'NNG'), ('정보', 'NNG'), ('제작', 'NNG'), ('이', 'JKS'), ('가능', 'XR'), ('하', 'XSA'), ('게', 'EC'), ('되', 'VV'), ('었', 'EP'), ('다', 'EF'), ('.', 'SF'), ('특히', 'MAG'), (',', 'SP'), ('농업', 'NNP'), ('환경', 'NNP'), ('모니터링', 'NNG'), ('을', 'JKO'), ('위하', 'VV'), ('아', 'EC'), ('작물', 'NNG'), ('생산', 'NNP'), ('지역', 'NNG'), ('의', 'JKG'), ('피', 'XPN'), ('복지', 'NNP'), ('도', 'JX'), ('생성', 'NNG'), ('에', 'JKB'), ('대하', 'VV'), ('ㄴ', 'ETM'), ('연구', 'NNG'), ('가', 'JKS'), ('활발히', 'MAG'), ('진행'

In [57]:
#Kkma
print("kkma", len(kkma.pos(text_img)))
print(kkma.pos(text_img))

kkma 6513
[('최근', 'NNG'), ('UAV', 'OL'), ('(', 'SS'), ('Unmanned', 'OL'), ('Aerial', 'OL'), ('Vehicle', 'OL'), (')', 'SS'), ('를', 'JKO'), ('이용', 'NNG'), ('하', 'XSV'), ('여', 'ECS'), ('고해상도', 'NNG'), ('영상', 'NNG'), ('을', 'JKO'), ('편리', 'NNG'), ('하', 'XSV'), ('게', 'ECD'), ('취득', 'NNG'), ('하', 'XSV'), ('ㄹ', 'ETD'), ('수', 'NNB'), ('있', 'VV'), ('게', 'ECD'), ('되', 'VV'), ('면서', 'ECE'), ('저', 'MDT'), ('비용', 'NNG'), ('으로', 'JKM'), ('소규모', 'NNG'), ('지역', 'NNG'), ('의', 'JKG'), ('관측', 'NNG'), ('및', 'MAG'), ('공간', 'NNG'), ('정보', 'NNG'), ('제작', 'NNG'), ('이', 'JKS'), ('가능', 'NNG'), ('하', 'XSV'), ('게', 'ECD'), ('되', 'VV'), ('었', 'EPT'), ('다', 'EFN'), ('.', 'SF'), ('특히', 'MAG'), (',', 'SP'), ('농업', 'NNG'), ('환경', 'NNG'), ('모니터링', 'NNG'), ('을', 'JKO'), ('위하', 'VV'), ('여', 'ECS'), ('작물', 'NNG'), ('생산', 'NNG'), ('지역', 'NNG'), ('의', 'JKG'), ('피복', 'NNG'), ('지도', 'NNG'), ('생성', 'NNG'), ('에', 'JKM'), ('대하', 'VV'), ('ㄴ', 'ETD'), ('연구', 'NNG'), ('가', 'JKS'), ('활발히', 'MAG'), ('진행', 'NNG'), ('되', 'XSV'), ('고', '

In [58]:
#from sklearn.feature_extract``ion.text import TfidfVectorizer

### 논문초록의 명사만 추출 (twitter, kkma)

In [106]:
#twitter와 kkma로 명사만 추출
def make_corpus_rm_stopwords_twitter(texts):
    corpus = []
    for s in texts:
        corpus.append([''.join(p) for p in twitter.nouns(s) ])
    return corpus
def make_corpus_rm_stopwords_kkma(texts):
    corpus = []
    for s in texts:
        corpus.append([''.join(p) for p in kkma.nouns(s)]) 
    return corpus

In [120]:
#'이미지'로 검색한 논문 초록 tokenizing
corpus_img_t = make_corpus_rm_stopwords_twitter(img)
corpus_img_k = make_corpus_rm_stopwords_kkma(img)

In [121]:
corpus_img_t[0] #twitter로 형태소 분리를 하여 명사만 추출한 경우

['최근',
 '를',
 '이용',
 '해상도',
 '영상',
 '취득',
 '수',
 '비용',
 '소규모',
 '지역',
 '관측',
 '및',
 '공간',
 '정보',
 '제작']

In [122]:
corpus_img_k[0]#kkma로 형태소 분리를 하여 명사만 추출한 경우

['최근',
 '이용',
 '고',
 '고해상도',
 '해상도',
 '영상',
 '편리',
 '취득',
 '수',
 '비용',
 '소규모',
 '지역',
 '관측',
 '공간',
 '공간정보',
 '정보',
 '제작',
 '가능']

In [123]:
#'음성'로 검색한 논문 초록 tokenizing
corpus_voice_t = make_corpus_rm_stopwords_twitter(voice)
corpus_voice_k = make_corpus_rm_stopwords_kkma(voice)

#'강화학습'로 검색한 논문 초록 tokenizing
corpus_RL_t = make_corpus_rm_stopwords_twitter(RL)
corpus_RL_k = make_corpus_rm_stopwords_kkma(RL)

#'텍스트'로 검색한 논문 초록 tokenizing
corpus_txt_t = make_corpus_rm_stopwords_twitter(txt)
corpus_txt_k = make_corpus_rm_stopwords_kkma(txt)

In [145]:
corpus_txt_t

[['연구',
  '글',
  '파일',
  '효율',
  '활용',
  '위해',
  '첨부',
  '표',
  '텍스트',
  '기반',
  '글',
  '파일',
  '웹브라우저',
  '바로',
  '확인',
  '수',
  '글',
  '파일',
  '파일',
  '변환',
  '소프트웨어',
  '구현'],
 ['관공서',
  '공공기관',
  '내부',
  '결재',
  '및',
  '공문',
  '글',
  '작성',
  '작성',
  '문서',
  '게시판',
  '통해',
  '공유'],
 ['문서', '대부분', '표', '텍스트', '구성', '대부분', '편집', '열람', '목적', '웹', '공유'],
 ['사용',
  '목적',
  '효율',
  '변환',
  '소프트웨어',
  '구현',
  '위해',
  '기존',
  '존재',
  '변환',
  '소프트웨어',
  '실행',
  '움',
  '변환',
  '오버헤드',
  '방법',
  '시한'],
 ['표',
  '텍스트',
  '특성',
  '이용',
  '서식',
  '파일',
  '패턴',
  '사용',
  '변환',
  '후',
  '구조',
  '단순화',
  '구현'],
 ['논문',
  '제시',
  '방법',
  '변환',
  '문서',
  '기존',
  '변환기',
  '생',
  '문서',
  '비교',
  '때',
  '태그',
  '속성',
  '사용',
  '대폭',
  '감소',
  '변화',
  '후',
  '개',
  '파일',
  '생',
  '변환',
  '결과',
  '단순화'],
 ['연구',
  '소셜',
  '네트워크',
  '서비스',
  '인',
  '플리커',
  '게시',
  '태깅',
  '사진',
  '데이터',
  '텍스트',
  '데이터',
  '활용',
  '우리나라',
  '방문객',
  '지역별',
  '이미지',
  '비교'],
 ['플리커',
  '데이터',
  '수집',
  '약',
  '데이터',
  '수집',

## Embedding

In [82]:
# module import
from gensim.models import Word2Vec, fasttext

# input format : [["my", "name", "is", "jamie"], ["jamie", "is", "cute"]]
# size : 임베딩 차원
# window : 윈도우 사이즈
# min_count : 고려할 단어의 최소 빈도 기준
# workers : cpu core 수
# iter : 반복 학습 횟수(epoch)
# sg : 0 -> CBOW, 1 -> Skip-Gram
# CBOW_model - Word2Vec(corpus, size=2, window=2, min_count=2, workers=1, iter=1000, sg=1)

#### '이미지'로 검색한 논문초록에서 '영상'이라는 단어와 유사도가 높은 단어들을 추출

In [149]:
# Skip-Gram으로 (twitter)
embedding_skipgram_model_img_t = Word2Vec(corpus_img_t, size=100, window = 3, min_count=3, workers=4, iter=100, sg=1)
print("skipgram: img_t  =  " ,embedding_skipgram_model_img_t.most_similar(positive=["영상"], topn=5))
# CBOW으로 (twitter)
embedding_cbow_model_img_t = Word2Vec(corpus_img_t, size=100, window = 3, min_count=3, workers=4, iter=100, sg=0)
print("cbow: img_t  =  " ,embedding_cbow_model_img_t.most_similar(positive=["영상"], topn=5))

# Skip-Gram으로 (kkma)
embedding_skipgram_model_img_k = Word2Vec(corpus_img_k, size=100, window = 3, min_count=3, workers=4, iter=100, sg=1)
print("skipgram: img_k  =  " ,embedding_skipgram_model_img_k.most_similar(positive=["영상"], topn=5))
# CBOW으로 (kkma)
embedding_cbow_model_img_k = Word2Vec(corpus_img_k, size=100, window = 3, min_count=3, workers=4, iter=100, sg=0)
print("cbow: img_k  =  " ,embedding_cbow_model_img_k.most_similar(positive=["영상"], topn=5))

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


skipgram: img_t  =   [('응답', 0.6927964091300964), ('정상파', 0.659285306930542), ('차원', 0.6505181789398193), ('점군', 0.648827075958252), ('효율', 0.6472326517105103)]


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


cbow: img_t  =   [('및', 0.9782615900039673), ('가지', 0.9782084226608276), ('가능성', 0.9751578569412231), ('다중', 0.9740633964538574), ('구축', 0.9736661314964294)]


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # Remove the CWD from sys.path while we load stuff.


skipgram: img_k  =   [('장', 0.8051372766494751), ('소규모', 0.7697581648826599), ('8', 0.7678431868553162), ('고해상도', 0.7494405508041382), ('위성', 0.7454818487167358)]
cbow: img_k  =   [('가능성', 0.9934436082839966), ('소규모', 0.992782711982727), ('관측', 0.9911009669303894), ('지역', 0.9904384613037109), ('피복', 0.9897207617759705)]


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  del sys.path[0]


In [ ]:
`

#### '음성'로 검색한 논문초록에서 '음성'이라는 단어와 유사도가 높은 단어들을 추출

In [143]:
# Skip-Gram으로 (twitter)
embedding_skipgram_model_voice_t = Word2Vec(corpus_voice_t, size=100, window = 3, min_count=3, workers=4, iter=100, sg=1)
print("skipgram: voice_t  =  " ,embedding_skipgram_model_voice_t.most_similar(positive=["음성"], topn=5))
# CBOW으로 (twitter)
embedding_cbow_model_voice_t = Word2Vec(corpus_voice_t, size=100, window = 3, min_count=3, workers=4, iter=100, sg=0)
print("cbow: voice_t  =  " ,embedding_cbow_model_voice_t.most_similar(positive=["음성"], topn=5))

# Skip-Gram으로 (kkma)
embedding_skipgram_model_voice_k = Word2Vec(corpus_voice_k, size=100, window = 3, min_count=3, workers=4, iter=100, sg=1)
print("skipgram: voice_k  =  " ,embedding_skipgram_model_voice_k.most_similar(positive=["음성"], topn=5))
# CBOW으로 (kkma)
embedding_cbow_model_voice_k = Word2Vec(corpus_voice_k, size=100, window = 3, min_count=3, workers=4, iter=100, sg=0)
print("cbow: voice_k  =  " ,embedding_cbow_model_voice_k.most_similar(positive=["음성"], topn=5))

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


skipgram: voice_t  =   [('간의', 0.8895830512046814), ('시각', 0.8524863719940186), ('위치', 0.8486015796661377), ('파악', 0.8421260714530945), ('구조', 0.8339105844497681)]


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


cbow: voice_t  =   [('간의', 0.9982788562774658), ('시각', 0.9980357885360718), ('전달', 0.9979836344718933), ('구조', 0.9979714155197144), ('가장', 0.9978514313697815)]


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # Remove the CWD from sys.path while we load stuff.


skipgram: voice_k  =   [('포함', 0.9614821672439575), ('방법', 0.9586842060089111), ('기능', 0.9510598182678223), ('제시', 0.9471137523651123), ('단어', 0.9463008642196655)]
cbow: voice_k  =   [('포함', 0.9991281628608704), ('기술', 0.9991122484207153), ('개선', 0.9990411996841431), ('전달', 0.9990339279174805), ('제안', 0.9990338683128357)]


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  del sys.path[0]


#### '강화학습'로 검색한 논문초록에서 '학습'이라는 단어와 유사도가 높은 단어들을 추출

In [142]:
# Skip-Gram으로 (twitter)
embedding_skipgram_model_RL_t = Word2Vec(corpus_RL_t, size=100, window = 3, min_count=3, workers=4, iter=100, sg=1)
print("skipgram: RL_t  =  " ,embedding_skipgram_model_RL_t.most_similar(positive=["학습"], topn=5))
# CBOW으로 (twitter)
embedding_cbow_model_RL_t = Word2Vec(corpus_RL_t, size=100, window = 3, min_count=3, workers=4, iter=100, sg=0)
print("cbow: RL_t  =  " ,embedding_cbow_model_RL_t.most_similar(positive=["학습"], topn=5))

# Skip-Gram으로 (kkma)
embedding_skipgram_model_RL_k = Word2Vec(corpus_RL_k, size=100, window = 3, min_count=3, workers=4, iter=100, sg=1)
print("skipgram: RL_k  =  " ,embedding_skipgram_model_RL_k.most_similar(positive=["학습"], topn=5))
# CBOW으로 (kkma)
embedding_cbow_model_RL_k = Word2Vec(corpus_RL_k, size=100, window = 3, min_count=3, workers=4, iter=100, sg=0)
print("cbow: RL_k  =  " ,embedding_cbow_model_RL_k.most_similar(positive=["학습"], topn=5))

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


skipgram: RL_t  =   [('강화', 0.9825229644775391), ('모델', 0.9749587774276733), ('심층', 0.962060272693634), ('하나', 0.9606121778488159), ('챗봇', 0.9601466655731201)]


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


cbow: RL_t  =   [('하나', 0.9991583228111267), ('강화', 0.9991542100906372), ('논문', 0.9991428852081299), ('모델', 0.9991282224655151), ('심층', 0.9991166591644287)]


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # Remove the CWD from sys.path while we load stuff.


skipgram: RL_k  =   [('강화', 0.9960197806358337), ('적용', 0.9950406551361084), ('강화학습', 0.994084358215332), ('에이전트', 0.9917963743209839), ('게임', 0.9916960597038269)]
cbow: RL_k  =   [('알고리즘', 0.999582052230835), ('논문', 0.9995130896568298), ('기존', 0.9995055794715881), ('강화', 0.9994878768920898), ('강화학습', 0.9994688630104065)]


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  del sys.path[0]


In [151]:
# Skip-Gram으로 (twitter)
embedding_skipgram_model_RL_t = Word2Vec(corpus_RL_t, size=100, window = 3, min_count=3, workers=4, iter=100, sg=1)
print("skipgram: RL_t  =  " ,embedding_skipgram_model_RL_t.most_similar(positive=["강화"], topn=5))
# CBOW으로 (twitter)
embedding_cbow_model_RL_t = Word2Vec(corpus_RL_t, size=100, window = 3, min_count=3, workers=4, iter=100, sg=0)
print("cbow: RL_t  =  " ,embedding_cbow_model_RL_t.most_similar(positive=["강화"], topn=5))

# Skip-Gram으로 (kkma)
embedding_skipgram_model_RL_k = Word2Vec(corpus_RL_k, size=100, window = 3, min_count=3, workers=4, iter=100, sg=1)
print("skipgram: RL_k  =  " ,embedding_skipgram_model_RL_k.most_similar(positive=["강화학습"], topn=5))
# CBOW으로 (kkma)
embedding_cbow_model_RL_k = Word2Vec(corpus_RL_k, size=100, window = 3, min_count=3, workers=4, iter=100, sg=0)
print("cbow: RL_k  =  " ,embedding_cbow_model_RL_k.most_similar(positive=["강화학습"], topn=5))

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


skipgram: RL_t  =   [('학습', 0.9825229644775391), ('챗봇', 0.9821451902389526), ('심층', 0.9752643704414368), ('구조', 0.9751850366592407), ('러닝', 0.9719106554985046)]


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


cbow: RL_t  =   [('심층', 0.9992510080337524), ('알고리즘', 0.9992199540138245), ('개발', 0.9991894960403442), ('모델', 0.9991734623908997), ('학습', 0.9991543292999268)]


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # Remove the CWD from sys.path while we load stuff.


skipgram: RL_k  =   [('강화', 0.997899055480957), ('알고리즘', 0.9947723150253296), ('학습', 0.9940844774246216), ('논문', 0.9937841892242432), ('적용', 0.9930306673049927)]
cbow: RL_k  =   [('알고리즘', 0.999488890171051), ('강화', 0.9994881749153137), ('학습', 0.9994688630104065), ('구현', 0.999459445476532), ('제안', 0.9994529485702515)]


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  del sys.path[0]


#### '텍스트'로 검색한 논문초록에서 '텍스트'이라는 단어와 유사도가 높은 단어들을 추출

In [146]:
# Skip-Gram으로 (twitter)
embedding_skipgram_model_txt_t = Word2Vec(corpus_txt_t, size=100, window = 3, min_count=3, workers=4, iter=100, sg=1)
print("skipgram: txt_t  =  " ,embedding_skipgram_model_txt_t.most_similar(positive=["텍스트"], topn=5))
# CBOW으로 (twitter)
embedding_cbow_model_txt_t = Word2Vec(corpus_txt_t, size=100, window = 3, min_count=3, workers=4, iter=100, sg=0)
print("cbow: txt_t  =  " ,embedding_cbow_model_txt_t.most_similar(positive=["텍스트"], topn=5))

# Skip-Gram으로 (kkma)
embedding_skipgram_model_txt_k = Word2Vec(corpus_txt_k, size=100, window = 3, min_count=3, workers=4, iter=100, sg=1)
print("skipgram: txt_k  =  " ,embedding_skipgram_model_txt_k.most_similar(positive=["텍스트"], topn=5))
# CBOW으로 (kkma)
embedding_cbow_model_txt_k = Word2Vec(corpus_txt_k, size=100, window = 3, min_count=3, workers=4, iter=100, sg=0)
print("cbow: txt_k  =  " ,embedding_cbow_model_txt_k.most_similar(positive=["텍스트"], topn=5))

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


skipgram: txt_t  =   [('이용', 0.911625862121582), ('마', 0.9072810411453247), ('이닝', 0.8942490816116333), ('플리커', 0.7899271845817566), ('대상', 0.7621303200721741)]


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


cbow: txt_t  =   [('이용', 0.9977251887321472), ('적용', 0.9946448802947998), ('및', 0.9946255683898926), ('추출', 0.9942023754119873), ('그', 0.9941701889038086)]


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # Remove the CWD from sys.path while we load stuff.


skipgram: txt_k  =   [('수행', 0.9383631944656372), ('이용', 0.9314905405044556), ('텍스트마이닝', 0.9198266267776489), ('마이닝', 0.9165320992469788), ('추출', 0.8939434289932251)]
cbow: txt_k  =   [('수행', 0.9990115165710449), ('구성', 0.9987563490867615), ('분석', 0.9986503720283508), ('이용', 0.998612105846405), ('방법론', 0.9985994696617126)]


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  del sys.path[0]
